## Wigets

In [45]:
from ipywidgets import IntSlider, Checkbox

In [46]:
slider = IntSlider()

In [47]:
slider.value

0

In [48]:
slider.value = 20

In [49]:
slider

IntSlider(value=20)

In [51]:
slider.value

66

In [52]:
checkbox = Checkbox(description="Check me")
checkbox

Checkbox(value=False, description='Check me')

In [53]:
checkbox.value = False

In [54]:
checkbox

Checkbox(value=False, description='Check me')

In [2]:
import os
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt

from dotenv import load_dotenv
from IPython.display import display
from ipywidgets import Video, Image

In [ ]:
load_dotenv()

tapo_username = os.getenv("TAPO_USERNAME")
tapo_password = os.getenv("TAPO_PASSWORD")
tapo_local_ip = os.getenv("TAPO_LOCAL_IP")
rtsp_port = 554

RTSP_URL = f"rtsp://{tapo_username}:{tapo_password}@{tapo_local_ip}:{rtsp_port}/stream1"
os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = "rtsp_transport;udp"

# print(RTSP_URL)

In [ ]:
import os
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt

from dotenv import load_dotenv
from IPython.display import display, clear_output
from ipywidgets import Image

load_dotenv()

tapo_username = os.getenv("TAPO_USERNAME")
tapo_password = os.getenv("TAPO_PASSWORD")
tapo_local_ip = os.getenv("TAPO_LOCAL_IP")
rtsp_port = 554

RTSP_URL = f"rtsp://{tapo_username}:{tapo_password}@{tapo_local_ip}:{rtsp_port}/stream1"
os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = "rtsp_transport;udp"

# Initialize Matplotlib figure
fig, ax = plt.subplots()
ax.set_title("FPS on every frame")
ax.set_ylabel("FPS")
ax.set_xlabel("Frame Number")

# Initialize OpenCV video capture
cap = cv2.VideoCapture(RTSP_URL)

# Track FPS
pTime = 0
frame_number = 0
frame_numbers_list = []
fps_list = []

# Initialize display widget
image_widget = Image()
display(image_widget)

while True:
    success, img = cap.read()
    if not success:
        print("Failed to read frame.")
        break

    # Resize frame
    img = cv2.resize(img, (640, 480))

    # Calculate FPS
    cTime = time.time()
    fps = 1 / (cTime - pTime) if pTime else 0
    pTime = cTime

    # Update FPS data
    frame_numbers_list.append(frame_number)
    fps_list.append(int(fps))
    frame_number += 1

    # Update FPS plot
    ax.clear()
    ax.plot(frame_numbers_list, fps_list, color="blue")
    ax.set_title("FPS on every frame")
    ax.set_ylabel("FPS")
    ax.set_xlabel("Frame Number")

    # Convert Matplotlib figure to image
    fig.canvas.draw()
    buffer, (width, height) = fig.canvas.print_to_buffer()
    plot = np.frombuffer(buffer, dtype=np.uint8).reshape((height, width, 4))  # RGBA

    # Remove alpha channel (Matplotlib uses RGBA, OpenCV uses BGR)
    plot = cv2.cvtColor(plot, cv2.COLOR_RGBA2BGR)

    # Combine video frame and FPS plot
    result_img = np.hstack([img, plot])

    # Encode frame for Jupyter display
    _, encoded_img = cv2.imencode(".png", result_img)
    image_widget.value = encoded_img.tobytes()

    # Exit on 'q' press
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


Image(value=b'')

[h264 @ 0x1401fd9c0] cabac decode of qscale diff failed at 78 23
[h264 @ 0x1401fd9c0] error while decoding MB 78 23, bytestream 233
[h264 @ 0x1136ecac0] cabac decode of qscale diff failed at 96 52
[h264 @ 0x1136ecac0] error while decoding MB 96 52, bytestream 1069


In [1]:
# Force TCP (more reliable), set small buffer
os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = "rtsp_transport;tcp"
cap = cv2.VideoCapture(RTSP_URL, cv2.CAP_FFMPEG)
# cap = cv2.VideoCapture(RTSP_URL) # UDP (faster, but less reliable)
# cap = cv2.VideoCapture(0) # Webcam


# cap.set(cv2.CAP_PROP_BUFFERSIZE, 2)  # Small buffer
cap.set(cv2.CAP_PROP_FPS, 15)  # Limit FPS for smooth playback

# Initialize Matplotlib figure
fig, ax = plt.subplots()
ax.set_title("FPS on every frame")
ax.set_ylabel("FPS")
ax.set_xlabel("Frame Number")

# Track FPS
pTime = 0
frame_number = 0
frame_numbers_list = []
fps_list = []

# Display widget
image_widget = Image()
display(image_widget)

try:
    while True:
        success, img = cap.read()
        if not success:
            print("⚠️ Failed to read frame. Check RTSP connection.")
            break

        img = cv2.resize(img, (640, 480))

        # Calculate FPS
        cTime = time.time()
        fps = 1 / (cTime - pTime) if pTime else 0
        pTime = cTime

        # Update FPS data
        frame_numbers_list.append(frame_number)
        fps_list.append(int(fps))
        frame_number += 1

        # Update FPS plot
        ax.clear()
        ax.plot(frame_numbers_list, fps_list, color="blue")
        ax.set_title("FPS on every frame")
        ax.set_ylabel("FPS")
        ax.set_xlabel("Frame Number")

        # Convert Matplotlib figure to image
        fig.canvas.draw()
        buffer, (width, height) = fig.canvas.print_to_buffer()
        plot = np.frombuffer(buffer, dtype=np.uint8).reshape((height, width, 4))  # RGBA
        plot = cv2.cvtColor(plot, cv2.COLOR_RGBA2BGR)

        # Combine video frame and FPS plot
        result_img = np.hstack([img, plot])

        # Encode frame for Jupyter display
        _, encoded_img = cv2.imencode(".png", result_img)
        image_widget.value = encoded_img.tobytes()

        # Add a small delay for smooth playback
        time.sleep(0.01)

except KeyboardInterrupt:
    print("\n✅ Stream stopped by user (Ctrl+C).")

finally:
    cap.release()
    cv2.destroyAllWindows()
    print("🎬 Stream closed.")

NameError: name 'os' is not defined